In [1]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

2219

In [3]:
with open(files[0], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\0001be30-1ef6-40d8-8f92-13ffbd093cf1.json'

In [5]:
json_data['data']['attributes']

{'id': '0001be30-1ef6-40d8-8f92-13ffbd093cf1'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Normal',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 2664924,
 'dateStarted': '2022-07-08T06:53:09.588+00:00',
 'rounds': 24,
 'isRanked': False,
 'queueId': 'unrated',
 'map': '2fb9a4fd-47b8-4e7d-a969-74b4046ebd53',
 'mapName': 'Breeze',
 'mapImageUrl': 'https://titles.trackercdn.com/valorant-api/maps/2fb9a4fd-47b8-4e7d-a969-74b4046ebd53/splash.png'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T18:11:36.1374295+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## round_summary

In [11]:
len(round_summary)

24

In [12]:
# round_summary[22]


In [13]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Red'

## player_summary

In [14]:
len(player_summary)

10

In [15]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [16]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'Hennessyy#AKA47'}

In [17]:
player_summary[0]['metadata']

{'partyId': '4155c5e2-bf8d-4736-acb1-8ec95e2576b1',
 'teamId': 'Blue',
 'agentKey': 'add6443a-41bd-e414-f6ad-e58d267f4e95',
 'agentName': 'Jett',
 'agentColor': '#98DCFF',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/add6443a-41bd-e414-f6ad-e58d267f4e95/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/jett_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None}}

In [18]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [19]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/0.png'},
 'value': 'Unranked',
 'displayValue': 'Unranked',
 'displayType': 'String'}

In [20]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 7621,
 'displayValue': '7,621',
 'displayType': 'Number'}

In [21]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 317.5416666666667,
 'displayValue': '318',
 'displayType': 'Number'}

## Win prediction data set creation

In [22]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']

In [23]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for i in roles_list:
            if i == agent_role:
                new_record[f'role_{i}'] = 1
            else:
                new_record[f'role_{i}'] = 0
        
        for i in all_agents_list:
            if i == agent:
                new_record[f'agent_{i}'] = 1
            else:
                new_record[f'agent_{i}'] = 0
            
        data.append(new_record)
            
    return data
        
        
processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary)
# processed_match_records[8]

In [24]:


def create_agent_row(agent_list, map_pick, game_win):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
        
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            
    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team))
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team))
    
    return [row1, row2]
    
    




In [25]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


(22230, 4438)

In [26]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'strawberry milk#우유갠디', 'count': 37},
 {'user': 'Thug Twerker#money', 'count': 27},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'Daddy Brim#OTP', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'Tan#0008', 'count': 23},
 {'user': 'Arowana#queer', 'count': 23},
 {'user': 'DaddyMingze#NA1', 'count': 22},
 {'user': 'No Heads Hawes#2025', 'count': 22},
 {'user': 'milktea#dad', 'count': 22},
 {'user': 'Swift Raptor#NA1', 'count': 22},
 {'user': 'Yor#rice', 'count': 21},
 {'user': 'hardstuckiron#3570', 'count': 21},
 {'user': 'DarkPegasis123#NA1', 'count': 21},
 {'user': '32Equr#6990', 'count': 21},
 {'user': 'bigmanduncachino#6969', 'count': 21},
 {'user': 'Baguette#sus', 'count': 21}]

In [27]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

4138

In [28]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,Yor#rice,2022-07-08T06:53:09.588+00:00,1,Red,Breeze,Unranked,5623,234.291667,0.833333,20,...,0,0,0,0,0,0,1,0,0,0
1,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
2,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
3,DarkPegasis123#NA1,2022-08-06T17:49:51.751+00:00,0,Blue,Pearl,Gold 3,2898,144.900000,0.400000,8,...,0,0,0,0,0,0,0,0,0,1
4,Apples#1706,2022-08-04T21:23:53.819+00:00,0,Blue,Pearl,Bronze 2,3300,137.500000,0.416667,10,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4133,fork#9925,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Bronze 1,2257,118.789474,0.421053,8,...,0,0,0,0,0,0,0,0,0,1
4134,Dark Dick#2499,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Unranked,7557,397.736842,1.368421,26,...,0,0,0,0,1,0,0,0,0,0
4135,Ham#6386,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Bronze 1,5296,331.000000,1.125000,18,...,0,1,0,0,0,0,0,0,0,0
4136,Hitchman224#1988,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Unranked,3455,215.937500,0.750000,12,...,0,0,0,0,0,0,0,0,0,0


In [29]:
all_records_df['name'].nunique()

294

In [30]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [31]:
all_records_df['map_pick'].value_counts(normalize=True)

Fracture    0.150556
Haven       0.146448
Ascent      0.145964
Breeze      0.139439
Bind        0.138231
Icebox      0.136539
Pearl       0.135331
Split       0.007492
Name: map_pick, dtype: float64

In [32]:
all_records_df['killsPerRound'].describe()

count    4138.000000
mean        0.807326
std         1.466178
min         0.000000
25%         0.500000
50%         0.666667
75%         0.900000
max        39.000000
Name: killsPerRound, dtype: float64

In [33]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 1      0.137023
Bronze 2      0.128806
Bronze 3      0.113098
Silver 1      0.090865
Silver 3      0.086757
Iron 3        0.084824
Unranked      0.070807
Silver 2      0.058482
Gold 1        0.046641
Iron 2        0.045433
Gold 2        0.041808
Gold 3        0.040358
Platinum 1    0.026825
Platinum 2    0.011600
Platinum 3    0.008942
Iron 1        0.007008
Diamond 1     0.000725
Name: rank, dtype: float64

In [34]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [35]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

len(training_players), len(val_players)

(220, 74)

In [36]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [37]:
# training_data_df.isna().sum()

In [38]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5060606060606061

In [39]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.2,4231.0,211.336018,0.767194,15.4,14.2,3.6,1.135754,2895.6,145.358210,5.8,2.2,1.0,0.6,0.0,1.6,5.4,9.0,9.6,2.8,0.286394,0.472816,0.480097,0.140259,0.6,0.6,2.4,3.4,47.500000,0.118139,0.168222,66.0,24.151914,65.347192,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2
1,0.0,4788.4,231.038518,0.812194,17.0,16.4,4.2,1.017333,3207.4,155.095710,6.8,2.0,1.0,0.6,0.2,1.8,6.0,8.4,9.2,3.0,0.298894,0.425316,0.430097,0.145259,0.6,0.6,2.6,3.4,67.500000,0.125639,0.168222,74.4,23.896284,66.347192,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2
2,0.0,5180.4,249.694013,0.841315,17.4,16.6,4.6,1.023766,3434.0,166.235270,7.0,1.6,1.4,0.6,0.2,2.2,7.8,8.0,9.0,2.8,0.371422,0.414327,0.431746,0.139764,0.6,0.6,3.4,2.8,75.555556,0.158057,0.147892,82.6,25.537150,68.819719,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2
3,0.2,5816.0,269.429842,0.930085,20.0,16.6,4.2,1.186266,3737.8,172.993559,7.4,2.4,1.6,0.6,0.2,2.4,8.2,8.6,11.2,3.2,0.370887,0.414862,0.513029,0.152598,0.6,0.6,4.2,2.4,86.984127,0.194420,0.116342,84.2,25.179449,71.333088,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
4,0.4,5863.8,274.063760,0.947629,20.2,16.2,3.6,1.229123,3801.2,177.853208,8.0,2.4,1.4,0.6,0.2,2.2,8.0,6.4,12.4,3.0,0.363869,0.301412,0.582620,0.143242,0.6,0.6,4.4,2.0,93.984127,0.206701,0.095874,88.8,25.604981,72.034842,0.0,0.0,0.6,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,1.0,5236.0,315.424309,1.145842,18.8,7.8,5.0,5.675000,3340.0,200.051218,5.8,3.0,1.0,0.8,0.2,2.0,4.4,10.4,12.8,1.2,0.250413,0.581223,0.784130,0.072328,0.0,0.0,2.8,0.6,187.000000,0.165439,0.034722,140.8,24.464506,83.230658,0.0,0.2,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2
0,0.8,1805.8,91.601015,0.322572,6.4,13.4,3.0,0.455821,1194.6,59.928866,4.2,1.2,0.0,0.0,0.0,0.0,23.2,18.4,18.2,2.0,1.329943,0.952700,0.915208,0.106103,2.8,2.8,0.2,1.2,20.000000,0.010526,0.060238,23.2,8.154920,57.740921,0.0,0

In [40]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
28,past_5_games_avg_esr,0.031538
16,past_5_games_avg_grenadeCasts,0.032011
31,past_5_games_avg_econRating,0.032053
20,past_5_games_avg_grenadeCastsPerRound,0.032444
21,past_5_games_avg_ability1CastsPerRound,0.033415
30,past_5_games_avg_firstDeathsPerRound,0.033518
22,past_5_games_avg_ability2CastsPerRound,0.033700
23,past_5_games_avg_ultimateCastsPerRound,0.033897
32,past_5_games_avg_hsAccuracy,0.035395
33,past_5_games_avg_kast,0.037518


In [41]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5136363636363637

In [42]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

columns_list = training_data_df.columns.tolist()

for i in columns_list:
    df_interaction[i] =  all_data_df[i]
    for j in columns_list:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_score_mul_past_5_games_avg_singleKills,past_5_games_avg_score_mul_past_5_games_avg_doubleKills,past_5_games_avg_score_mul_past_5_games_avg_tripleKills,past_5_ga

In [43]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(20)

,column,slope,intercept,rvalue,r2,pvalue,stderr
352,past_5_games_avg_assists_mul_past_5_games_avg_econRating,25.072011,265.026514,0.103938,0.010803,7.417092e-08,4.646482
330,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,52.859350,622.261162,0.095045,0.009034,8.735235e-07,10.722366
328,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.535167,5.015811,0.093261,0.008698,1.396847e-06,0.110653
329,past_5_games_avg_assists_mul_past_5_games_avg_damage,1094.975049,12680.415438,0.088174,0.007775,5.086433e-06,239.572688
349,past_5_games_avg_assists_mul_past_5_games_avg_esr,40.469686,278.616692,0.084158,0.007082,1.345028e-05,9.280329
381,past_5_games_avg_kdRatio_mul_past_5_games_avg_singleKills,0.532196,6.276808,0.082965,0.006883,1.780821e-05,0.123808
546,past_5_games_avg_singleKills_mul_past_5_games_avg_econRating,21.030420,333.402956,0.078539,0.006168,4.883263e-05,5.169953
331,past_5_games_avg_assists_mul_past_5_games_avg_singleKills,2.039737,27.816212,0.078139,0.006106,5.336996e-05,0.504021
453,past_5_games_avg_damage_mul_past_5_games_avg_kast,11118.738729,189268.375080,0.077933,0.006074,5.584894e-05,2754.740663
332,past_5_games_avg_assists_mul_past_5_games_avg_doubleKills,1.016645,11.778049,0.077016,0.005931,6.830061e-05,0.254898


In [44]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
8,past_5_games_avg_damage,124.571276,2636.045050,0.076563,5.861821e-03,0.000075,31.419187
7,past_5_games_avg_kdRatio,0.076495,1.071961,0.073343,5.379232e-03,0.000150,0.020145
9,past_5_games_avg_damagePerRound,5.303629,131.711114,0.072944,5.320891e-03,0.000163,1.404405
6,past_5_games_avg_assists,0.236799,4.704017,0.070751,5.005669e-03,0.000255,0.064659
2,past_5_games_avg_scorePerRound,-54.406608,263.306804,-0.066368,4.404686e-03,0.000603,15.841836
31,past_5_games_avg_econRating,2.377964,57.089383,0.066283,4.393385e-03,0.000613,0.693297
10,past_5_games_avg_singleKills,0.178712,5.742324,0.066274,4.392243e-03,0.000614,0.052110
3,past_5_games_avg_killsPerRound,-0.169554,0.888100,-0.065194,4.250270e-03,0.000753,0.050263
28,past_5_games_avg_esr,5.581753,59.597968,0.063289,4.005508e-03,0.001072,1.704669
11,past_5_games_avg_doubleKills,0.115078,2.463415,0.057125,3.263229e-03,0.003161,0.038952



## Agent pick prediction data set creation

In [ ]:
all_records, all_agent_records = get_all_processed_data()


In [ ]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [ ]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

In [ ]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

In [ ]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

In [ ]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

In [ ]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

In [ ]:
rf = LogisticRegression()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

In [ ]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

In [ ]:
from sklearn.decomposition import PCA

rf = LogisticRegression()
pca = PCA()

x_pca = pca.fit_transform(train_x)
x_val_pca = pca.transform(val_x)

rf.fit(x_pca, train_y)

preds = rf.predict(x_val_pca)

accuracy_score(preds, val_y)

In [ ]:
model= RandomForestClassifier(n_estimators=100, max_features = .9, max_samples=.5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

In [ ]:
# model.predict(val_x)

In [ ]:
# model.predict_proba(val_x)[0][-1]

In [ ]:
model= RandomForestClassifier(n_estimators=1000)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

In [ ]:
model= RandomForestClassifier(n_estimators=1000)


train_df, val_df = train_test_split(all_agent_records_df)

all_x = all_agent_records_df.drop('game_win', axis = 1)
all_y = all_agent_records_df['game_win']

model.fit(all_x, all_y)


In [ ]:
def predict_best_lineup(model, map_pick, current_agent_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df = features_df.drop('game_win', axis = 1)
    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index
    preds = model.predict_proba(features_df)

    inputs_df['win_prob'] = model.predict_proba(features_df)[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = ['Phoenix',
                     'Reyna',
                      'Raze',
                     ]

predict_best_lineup(model, map_pick, current_agent_list)

In [ ]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [ ]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model, map_pick, current_agent_list).tail()